# cuPauliProp 튜토리얼 - 2큐비트 시스템

**목표**: H(0) → CNOT(0,1) 회로 적용 후 Z₀의 기댓값 계산

**Heisenberg Picture**: Observable을 propagate하여 $\langle 0|U^\dagger Z_0 U|0\rangle$ 계산

**예상 결과**: 0 (벨 상태에서 Z₀의 기댓값)

## Step 1: 라이브러리 임포트

In [1]:
import numpy as np
import cupy as cp
from cuquantum.bindings import cupauliprop as cupp

print("✓ cuPauliProp 라이브러리 로드 완료")
print(f"  CuPy: {cp.__version__}")

✓ cuPauliProp 라이브러리 로드 완료
  CuPy: 13.6.0


## Step 2: Handle 생성

Handle = cuPauliProp의 작업 공간 ID  

하나의 handle 내에서:
1. observable을 정의하고 
2. 게이트를 생성하며
3. Pauli propagation 및
4. 기댓값 계산까지 다 이루어짐

In [12]:
handle = cupp.create()
print("✓ Handle 생성")

# 만약 여러 작업 공간을 만들고 싶다면 변수 명 다르게 호출. 
# handle1 = cupp.create()
# handle2 = cupp.create()
# handle3 = cupp.create()

✓ Handle 생성


## Step 3: 초기 Observable 설정 (Z₀)  

num_packed_ints 는 큐비트를 효율적으로 담기 위함. int 1개는 ```uint64```로 64비트로 이루어져 있으며, 각 비트 자리수가 큐비트 인덱스라 보면 됨  
* ex 00....100 = 2번 큐비트 (0~63번 이라고 할 때)
* 0b01 = 0b1 = 비트 0이 1 → 큐빗 0
* 0b10 = 비트 1이 1 → 큐빗 1
* 0b100 = 비트 2가 1 → 큐빗 2
* 0b11 = 비트 0과 1이 1 → 큐빗 0과 1

Pauli 연산자를 비트로 인코딩:
- X mask: X 연산이 있는 큐빗
- Z mask: Z 연산이 있는 큐빗

In [13]:
num_qubits = 2
num_packed_ints = cupp.get_num_packed_integers(num_qubits)

print(f"큐빗 수: {num_qubits}")
print(f"Packed integers: {num_packed_ints}")

# Z₀: qubit 0에만 Z 연산
# Pauli 데이터 구조: [X_mask, Z_mask] 순서
# d_input_pauli = cp.array([0b00, 0b01], dtype=cp.uint64)  # 얘는 X가 아무한테도 안 씌어져 있고, Z가 0번 큐비트에만 씌어져 있다는 뜻, 아무것도 없으면 0으로 두어도 됨
# d_input_pauli = cp.array([0b01, 0b01], dtype=cp.uint64) # 얘는 (XZ)_0 = -i(Y)_0 이기도 함
# d_input_pauli = cp.array([0b10, 0b01], dtype=cp.uint64) # 얘는 Z_0 X_1 
d_input_pauli = cp.array([0b00, 0b10], dtype=cp.uint64) # 얘는 Z_1
d_input_coef = cp.array([1.0], dtype=cp.float64)

# 만약 해밀토니안 H가 선형 합이면? 
# d_input_pauli = cp.array([
#                         0b00, 0b01,  # Z0: [X_mask, Z_mask]
#                         0b10, 0b10   # X1: [X_mask, Z_mask]
#                     ], dtype=cp.uint64)
# d_input_coef = cp.array([1.0, 1.0], dtype=cp.float64)  # 계수: 1×Z0 + 0.3×Z₁
# 이제 한 번의 propagation으로 (Z0 + 0.3×Z₁) 계산

큐빗 수: 2
Packed integers: 1


## Step 4: Pauli Expansion 생성  

Pauli Expansion: 관측자를 GPU에서 처리할 수 있는 자료 구조로 만드는 과정임
* 예) Z_0 + 2X_1를 GPU 메모리의 버퍼로 변환하는 것

Buffer 기반 expansion 생성

In [14]:
max_terms = 50 
# 최대 몇 개의 pauli 항을 저장할지 == 트리로 나타냈을 때 리프의 개수
# 당연히 초기 관측자으 항 개수보다 많아야 함. non-clifford는 분기시켜서 2배씩 증가하기때문에 넉넉히 설정해야됨

########################################################################################
# 여기는 GPU 메모리에 저장공간을 확보하고 초기 데이터를 넣는 과정으로 아래의 수식을 앞으로도 그대로 쓰면 됨

# Input expansion buffer (초기 데이터: Z₀)
d_input_pauli_buffer = cp.zeros(2 * num_packed_ints * max_terms, dtype=cp.uint64) 
# 2 = X_mask, Z_mask / num_packed_ints = 큐빗 수에 따른 packed integer 개수 / max_terms = 저장공간 확보할 최대 항 개수

d_input_pauli_buffer[:len(d_input_pauli)] = d_input_pauli # 초기 데이터 복사, 나머지는 0으로 채워짐
# 애는 앞에서부터 2개씩이 각 큐비트를 의미하는 듯 만약 0번 큐비트에 Z만 있다면 전체 데이터는 [0,1,0,0,0,...] 이런 식
# H가 Z_0 + X_1 이면 [0,1,1,0,0,...] / (XZ)_0 = -i(Y)_0 이면 [1,1,0,0,0,...] 

d_input_coef_buffer = cp.zeros(max_terms, dtype=cp.float64) # 얘는 딱 알겠지? 걍 계수 버퍼
d_input_coef_buffer[:len(d_input_coef)] = d_input_coef

# 이 계산식도 그대로  사용해야 하는데 GPU API 설계가 비트가 아니라 바이트를 따르기 때문
pauli_buffer_size = 2 * num_packed_ints * max_terms * 8  # bytes
coef_buffer_size = max_terms * 8
########################################################################################

In [15]:
# 여기가 핵심 부분
input_expansion = cupp.create_pauli_expansion(
    handle,                           # Step 2의 handle
    num_qubits,                       # 큐빗 수
    d_input_pauli_buffer.data.ptr,   # Pauli 데이터 포인터
    pauli_buffer_size,                # 메모리 크기 (bytes)
    d_input_coef_buffer.data.ptr,    # 계수 데이터 포인터
    coef_buffer_size,                 # 메모리 크기 (bytes)
    1,                                # CUDA_R_64F (float64) - 고정
    len(d_input_coef),           # ← 수동: 초기 term 개수 (Step 3에서 만든 개수) 
    1, 1                              # sorted, unique - 보통 1, 1로 고정
)
# 여기서 8번째 인수는 관측자의 항의 개수에 따라 수동 조절을 해야 함 -> len(d_input_coef)로 두면 자동으로 될 듯
print(f"✓ Input expansion: {max_terms}개 capacity")

# Output expansion (빈 buffer)
d_output_pauli_buffer = cp.zeros(2 * num_packed_ints * max_terms, dtype=cp.uint64)
d_output_coef_buffer = cp.zeros(max_terms, dtype=cp.float64)

# 얘는 Gate 적용 결과를 받을 공간. PauliProp은 입력과 출력이 다르므로 분리해야 함
output_expansion = cupp.create_pauli_expansion(
    handle,
    num_qubits,
    d_output_pauli_buffer.data.ptr,
    pauli_buffer_size,
    d_output_coef_buffer.data.ptr,
    coef_buffer_size,
    1,  # CUDA_R_64F
    0,  # 초기 term 0개서부터 시작해야 함
    0, 0
)
print(f"✓ Output expansion: {max_terms}개 capacity")

✓ Input expansion: 50개 capacity
✓ Output expansion: 50개 capacity


## Step 5: Workspace 준비

GPU 계산용 임시 메모리  
Observable Z₀  
  → RY gate 적용  
     ├─ 중간 계산 결과 저장 (임시) ← Workspace에 저장  
     └─ 최종 결과 도출  

분기 과정에서:  
* 중간 계산 결과들을 어디에 저장?  
* GPU 임시 메모리(SCRATCH)가 필요!   
 
SCRATCH 메모리 없이는 불가능한 이유:  
* 모든 중간 계산을 CPU로 옮길 수 없음 (너무 느림)
* GPU 메모리에 계산 결과를 임시로 보관해야 함
* 이것이 바로 SCRATCH의 역할


In [16]:
workspace = cupp.create_workspace_descriptor(handle)
# PauliProp은 내부적으로 작업 공간이 필요함. 미리 할당해두기

workspace_size = 100 * 1024 * 1024  # 100 MB <- 회로가 커지면 더욱 더 많이 필요함. workspace_size = circuit_depth × max_terms × 1 MB (대략적)
d_workspace = cp.cuda.alloc(workspace_size) # GPU 메모리에 할당, d_workspace에는 GPu 메모리 포인터 객체가 할당됨

cupp.workspace_set_memory( 
    handle,
    workspace, # 할당한 메모리를 지정
    0,  # CUPAULIPROP_MEMSPACE_DEVICE 0이면 GPU, 1이면 CPU라고 함
    0,  # CUPAULIPROP_WORKSPACE_SCRATCH / SCRATCH:는 임시 작업 메모리인데, 아직 초기 버전에서는 0밖에 없는 듯
    d_workspace.ptr, # GPU 메모리 포인터 주고
    workspace_size  # 사이즈 주고
)

print(f"✓ Workspace: {workspace_size / 1024 / 1024:.1f} MB")

✓ Workspace: 100.0 MB


## Step 6: 양자 회로 구성  

#### P = X, Y, Z 또는 tensor product
```
operator = cupp.create_pauli_rotation_gate_operator(
    handle,
    angle,                  # 회전 각도 (라디안)
    num_qubits,            # gate가 작용하는 큐비트 수
    qubits.ctypes.data,    # 큐비트 인덱스
    paulis.ctypes.data     # Pauli 종류 (0=I, 1=X, 2=Y, 3=Z)
)
```

#### Clifford Gate (CNOT, CZ, S, H 등)
```
qubits = np.array([control, target], dtype=np.int32)
operator = cupp.create_clifford_gate_operator(
    handle,
    gate_kind,             # 0=CX, 1=CY, 2=CZ, 3=S, 4=Sdg, ...
    qubits.ctypes.data     # 큐비트 인덱스
)
```

**Clifford Gate 종류:**
- `0`: CX (CNOT)
- `1`: CY
- `2`: CZ
- `3`: S
- `4`: S†
- `5`: H (Hadamard)
- `6`: X
- `7`: Y
- `8`: Z


In [17]:
circuit = []
PI = np.pi

print("회로 구성 (다양한 rotation으로 임의 기댓값 생성):")


# P = X, Y, Z 또는 tensor product
# qubits = np.array([0, 1], dtype=np.int32)
# paulis = np.array([1, 2], dtype=np.int32)  # X⊗Y
# operator = cupp.create_pauli_rotation_gate_operator(
#     handle,
#     angle,                  # 회전 각도 (라디안)
#     num_qubits,            # gate가 작용하는 큐비트 수
#     qubits.ctypes.data,    # 큐비트 인덱스
#     paulis.ctypes.data     # Pauli 종류 (0=I, 1=X, 2=Y, 3=Z)
# )


qubit_0 = np.array([0], dtype=np.int32)
qubit_1 = np.array([1], dtype=np.int32)
# qubit_8 = np.array([8], dtype=np.int32)  # 큐빗 8번

# 얘네는 고정임
pauli_x = np.array([1], dtype=np.int32)  # CUPAULIPROP_PAULI_X 1 
pauli_y = np.array([2], dtype=np.int32)  # CUPAULIPROP_PAULI_Y 2 
pauli_z = np.array([3], dtype=np.int32)  # CUPAULIPROP_PAULI_Z 3


ry1 = cupp.create_pauli_rotation_gate_operator(
    handle, PI/4, 1, qubit_1.ctypes.data, pauli_y.ctypes.data
)



# print("  3. CNOT(0 → 1)")
# cnot_qubits = np.array([0, 1], dtype=np.int32) # 순서대로 [컨트롤 큐비트, 타겟 큐비트]
# cnot = cupp.create_clifford_gate_operator(
#     handle,
#     0,  # CUPAULIPROP_CLIFFORD_GATE_KIND_CX
#     cnot_qubits.ctypes.data
# )


circuit.append(ry1)



print(f"\n✓ 총 {len(circuit)}개 게이트")
print(f"✓ Non-Clifford gate 포함 → Pauli 항 증가 예상")

회로 구성 (다양한 rotation으로 임의 기댓값 생성):

✓ 총 1개 게이트
✓ Non-Clifford gate 포함 → Pauli 항 증가 예상


## Step 7: Pauli Propagation (Heisenberg Picture)

게이트를 **역순**으로 **adjoint** 적용: $U^\dagger Z_0 U$  

In [18]:
print("Pauli propagation (정방향!):")

current_input = input_expansion # 초기 관측자
current_output = output_expansion # 결과 저장용

# 올바른 이해:
# State: |ψ⟩ = U|0⟩ (예: RY|0⟩)
# Observable: O (예: X_1)
# 
# 기댓값: ⟨ψ|O|ψ⟩ = ⟨0|U† O U|0⟩
#
# trace_with_zero_state는 ⟨0|·|0⟩를 계산하므로
# O' = U† O U를 먼저 만들어야 함
#
# 회로 [RY]에서 U = RY이므로
# U† O U = RY† O RY
#
# API가 adjoint=0일 때 G O G†를 계산하면:
#   RY를 adjoint=0으로 적용 → RY X_1 RY† (틀림!)
# API가 adjoint=1일 때 G† O G를 계산하면:
#   RY를 adjoint=1으로 적용 → RY† X_1 RY (맞음!)

# 역순으로 각 게이트를 적용 (Heisenberg picture)
for i in range(len(circuit) - 1, -1, -1):
    gate = circuit[i]
    
    num_terms = cupp.pauli_expansion_get_num_terms(handle, current_input)
    
    input_view = cupp.pauli_expansion_get_contiguous_range(
        handle, current_input, 0, num_terms
    )

    cupp.pauli_expansion_view_compute_operator_application(
        handle,
        input_view,     # 입력 관측자
        current_output, # 출력 결과
        gate,           # 게이트
        1,              # adjoint=0 → G O G†
        0,              # sort=False
        0,              # keep_duplicates=False
        0,              # no truncation
        None,           # truncation strategy
        workspace       # GPU 임시 메모리
    )
    
    cupp.destroy_pauli_expansion_view(input_view)
    
    # Swap
    current_input, current_output = current_output, current_input
    
    if i % 2 == 0 or i == len(circuit) - 1:
        new_num_terms = cupp.pauli_expansion_get_num_terms(handle, current_input)
        print(f"  Gate {i+1}: {new_num_terms}개 항")

final_num_terms = cupp.pauli_expansion_get_num_terms(handle, current_input)
print(f"\n✓ 최종 expansion: {final_num_terms}개 Pauli 항")

Pauli propagation (정방향!):
  Gate 1: 1개 항

✓ 최종 expansion: 1개 Pauli 항


In [19]:
# Step 8 앞부분에 추가

# 최종 expansion 내용 확인
print("\n=== 최종 Pauli Expansion 디버깅 ===")
final_num_terms = cupp.pauli_expansion_get_num_terms(handle, current_input)
print(f"항 개수: {final_num_terms}")

# 계수 확인
# 문서: "X component(s) of each string placed before the corresponding Z component(s)"
# → [X_term0, Z_term0, X_term1, Z_term1, ...]
if current_input == input_expansion:
    coef_data = d_input_coef_buffer[:final_num_terms].get()
    pauli_data = d_input_pauli_buffer[:final_num_terms*2*num_packed_ints].get()
else:
    coef_data = d_output_coef_buffer[:final_num_terms].get()
    pauli_data = d_output_pauli_buffer[:final_num_terms*2*num_packed_ints].get()

for i in range(final_num_terms):
    # 올바른 레이아웃: [X0, X1, ..., Z0, Z1, ...]
    x_mask = pauli_data[i]
    z_mask = pauli_data[final_num_terms + i]
    coef = coef_data[i]
    print(f"  Term {i}: X={x_mask:04b} Z={z_mask:04b} coef={coef:.6f}")

print("="*50)


=== 최종 Pauli Expansion 디버깅 ===
항 개수: 1
  Term 0: X=0000 Z=0010 coef=1.000000


## Step 8: 기댓값 계산

$\langle 0|U^\dagger O U|0\rangle$ 계산

In [20]:
final_num_terms = cupp.pauli_expansion_get_num_terms(handle, current_input)

final_view = cupp.pauli_expansion_get_contiguous_range(
    handle, current_input, 0, final_num_terms
)

result = np.array([0.0], dtype=np.float64)
cupp.pauli_expansion_view_compute_trace_with_zero_state(
    handle,
    final_view,
    result.ctypes.data,
    workspace
)

cupp.destroy_pauli_expansion_view(final_view)

expectation = result[0]  # 배열이 아닌 스칼라 값 추출

print("=" * 60)
print(f"Expectation = {expectation:.6f}")
print("=" * 60)


# Step 9: 리소스 해제
cupp.destroy_workspace_descriptor(workspace)
cupp.destroy(handle)

print("✓ 모든 리소스 해제 완료")

Expectation = 1.000000
✓ 모든 리소스 해제 완료


---

## 요약

### 핵심 개념

1. **Packed Integers**: Pauli 연산자를 비트로 인코딩 (X mask, Z mask)
2. **Pauli Expansion**: Observable을 Pauli 항들의 합으로 표현
3. **Heisenberg Picture**: $U^\dagger O U$ 계산을 위해 게이트를 역순 adjoint 적용
4. **Workspace**: GPU 임시 메모리

### Pauli Propagation 과정

```
Z₀ --[CNOT†]--> Z₀ --[RZ†]--> Z₀ --[RY†]--> X₀ --[RZ†]--> X₀
```

최종: X₀X₁ (대략적으로)

### 결과 해석

- 회로: H(0) → CNOT(0,1)
- 최종 상태: 벨 상태 $(|00\rangle + |11\rangle)/\sqrt{2}$
- Z₀ 기댓값: 0 (대칭성)

### 장점

- **GPU 가속**: CuPy로 GPU 메모리 직접 관리
- **효율성**: 상태벡터 대신 Pauli 연산자로 계산
- **확장성**: 수백 큐빗 시스템에도 적용 가능

---

## 2큐비트 Observable 테스트 (ZZ, XZ 등)

기존 코드는 Z₀ (single qubit)만 사용했습니다. 이제 2큐비트 observable을 테스트해봅시다.

### Observable 인코딩 규칙

2큐비트 시스템에서 Pauli observable 인코딩:

| Observable | X mask | Z mask | 설명 |
|------------|--------|--------|------|
| I (identity) | 0b00 | 0b00 | 아무것도 없음 |
| Z₀ | 0b00 | 0b01 | qubit 0에 Z |
| Z₁ | 0b00 | 0b10 | qubit 1에 Z |
| **Z₀Z₁ (ZZ)** | 0b00 | 0b11 | 둘 다 Z |
| X₀ | 0b01 | 0b00 | qubit 0에 X |
| X₁ | 0b10 | 0b00 | qubit 1에 X |
| **X₀Z₁ (XZ)** | 0b01 | 0b10 | q0=X, q1=Z |
| **Z₀X₁ (ZX)** | 0b10 | 0b01 | q0=Z, q1=X |
| **X₀X₁ (XX)** | 0b11 | 0b00 | 둘 다 X |
| Y₀ = iXZ | 0b01 | 0b01 | qubit 0에 Y |

**핵심**: 비트 위치 = 큐빗 인덱스

In [46]:
# Step 8에 대체 코드 추가

print("\n=== 올바른 기댓값 계산 (수동) ===")

# 최종 expansion 데이터
if current_input == input_expansion:
    coef_data = d_input_coef_buffer[:final_num_terms].get()
    pauli_data = d_input_pauli_buffer[:final_num_terms*2*num_packed_ints].get()
else:
    coef_data = d_output_coef_buffer[:final_num_terms].get()
    pauli_data = d_output_pauli_buffer[:final_num_terms*2*num_packed_ints].get()

manual_exp = 0.0

for i in range(final_num_terms):
    x_mask = pauli_data[2*num_packed_ints*i]
    z_mask = pauli_data[2*num_packed_ints*i + num_packed_ints]
    coef = coef_data[i]
    
    # |0⟩ 상태에서 Pauli 기댓값
    if x_mask == 0 and z_mask == 0:
        # Identity
        val = 1.0
    elif x_mask == 0:
        # Z만 있음: Z|0⟩ = |0⟩
        val = 1.0
    else:
        # X 또는 Y 있음: 비대각
        val = 0.0
    
    manual_exp += coef * val
    print(f"  항 {i}: coef={coef:7.4f} × {val:.1f} = {coef*val:7.4f}")

print(f"\n수동 계산: {manual_exp:.6f}")
print(f"CUDA-Q:   1.707107")
print(f"API 결과: {expectation:.6f}")

if abs(manual_exp - 1.0) < 0.01:
    print("\n⚠️  cuPauliProp의 compute_trace_with_zero_state는")
    print("   X/Y 항을 올바르게 처리하지 못합니다!")
    print("   → 문서 확인 필요 또는 다른 API 사용 필요")


=== 올바른 기댓값 계산 (수동) ===
  항 0: coef= 1.0000 × 1.0 =  1.0000
  항 1: coef= 0.7071 × 0.0 =  0.0000
  항 2: coef=-0.7071 × 0.0 = -0.0000

수동 계산: 1.000000
CUDA-Q:   1.707107
API 결과: 1.000000

⚠️  cuPauliProp의 compute_trace_with_zero_state는
   X/Y 항을 올바르게 처리하지 못합니다!
   → 문서 확인 필요 또는 다른 API 사용 필요


In [45]:
import cudaq
from cudaq import spin

H = spin.z(0) + spin.x(1)

@cudaq.kernel
def circuit():
    q = cudaq.qvector(2)
 
    ry( np.pi / 4, q[1])

cudaq.observe(circuit, H).expectation()

1.707106813788414

In [ ]:

# Step 1: CUDA-Q를 이용한 검증
print("\n" + "="*60)
print("CUDA-Q를 이용한 검증 (참값)")
print("="*60)

import cudaq
from cudaq import spin
import numpy as np

# Z_1 observable
H_z1 = spin.z(1)

# 회로: RY(π/4)를 큐비트 1에 적용
@cudaq.kernel
def circuit_z1():
    q = cudaq.qvector(2)
    ry(np.pi / 4, q[1])

result_cudaq = cudaq.observe(circuit_z1, H_z1)
exp_z1_cudaq = result_cudaq.expectation()

print(f"Observable: Z_1")
print(f"Circuit: RY(π/4) on qubit 1")
print(f"State: |00⟩ (ground state)")
print(f"\nExpected: cos(π/4) × 1 + sin(π/4) × 0")
print(f"        = {np.cos(np.pi/4):.6f} × 1 + {np.sin(np.pi/4):.6f} × 0")
print(f"        = {np.cos(np.pi/4):.6f}")
print(f"\nCUDA-Q Result: {exp_z1_cudaq:.6f}")
print("="*60)
